## Generating Instances with `pysa-genmci`

PySA-McEliece generates scaled down problem instances of the McEliece cryptosystem as benchmark cases for combinatorial optimization algorithms and devices. Each instance is defined by a public key matrix and ciphertext. The public key is created from a random Goppa code and the ciphertext is encoded from a random plaintext string and a random weight `t` error.

The recommended way to generate instances is to use the `pysa-mceliece` command line interface. Its required arguments and some useful flags are

```
pysa-genmci n t NUM_INST PUB_DIR PRIV_DIR [--nieder] [--seed 1234]...
```

where `n` is the length of the code, `t` is the correctable error distance, `NUM_INST` is the number of instances to generate, and `PUB_DIR` and `PRIV_DIR` are the destination directories for public and private information. 
If the flag `--nieder` is passed, the problem is formulated in its Niederreiter dual, which uses the parity check matrix instead of the generator matrix. The flag `--seed` passes an initial seed to the random number generators to allow instances to be reproducibly generated.

As a first example, we can generate a small McEliece instance as follows:

In [37]:
! pysa-genmci 16 3 1 mci_example_16-3_pub mci_example_16-3_priv  --seed 1234 

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


This will generate one instance with a code space of 16 bits and 3 error bits. By default, a Goppa code is over the smallest finite field $\mathbb{F}_2^m$ that will contain the code space, so $m=4$ in this case. The dimension of a Goppa code is $k=n-tm$, so we have generated a code with dimension $k=4$.
Reflecting all of these parameters, each file in the public and private directory is prefixed with 
`mci_n16_t3_m4_k4`.

In the private directory, the `_plain.txt` files contain 3 lines: The binary string of the $k$-bit plain text, the hex representation of the plain text, and the message encoded with the generator matrix prior to applying a weight $t$ random error.
The `_errs.txt` files contain the list of bit indices (starting from 0) that the random error flips.

In [9]:
! cat mci_example_16-3_priv/mci_n16_t3_m4_k4_0_plain.txt
! echo
! cat mci_example_16-3_priv/mci_n16_t3_m4_k4_0_errs.txt

0 0 1 1 
0C 
1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 

6 13 14 

The public directory contains the problem instance in various solver format.
The MLD format is described in `README.md`. For McEleiece instances, each line after the header lists the non-zero entries in each row of the generator matrix transpose.

In [11]:
! cat ./mci_example_16-3_pub/mci_n16_t3_m4_k4_0_mld.txt

g 4 16 1 3
1 3 
-1 
2 3 
1 3 
-2 3 4 
-1 
-1 4 
-1 3 4 
2 4 
-1 3 4 
2 3 
2 4 
-1 2 
1 2 3 4 
-1 2 3 
-2 3 4 


The parity check decoding (PCD) dual (i.e. the Niederreiter dual) of the McEliece instance is also generated, which uses the parity check matrix and is the natural input to various decoding algorithms.
These are written to `_pcd.txt`. (Note, if the `--nieder` flag is passed, only the Niederreiter dual is generated and written to the  `_mld.txt` file).

In [12]:
! cat ./mci_example_16-3_pub/mci_n16_t3_m4_k4_0_pcd.txt

h 16 12 1 3
-1 4 
-2 6 
2 3 5 7 
-1 3 5 8 
-1 2 5 9 
-1 3 5 10 
-3 11 
-1 2 5 12 
-1 3 13 
2 5 14 
2 3 15 
-5 16 


## Internal API for PySA-McEliece

(Note: The internal Python API for this package is not stable and subject to change)

The `pysa-genmci` CLI integrates various of the submodules under the `mceliece` package into a single instance generator pipeline. A more flexible workflow can be created by directly importing from these submodules.

Goppa codes are encapsulated by the `mceliece.goppa.GoppaCode` class. A random Goppa code can be conveniently created using `generate_random_goppa(n, t)`, which generates the required constructor data using the `galois` package.

The `mceliece.generator.McElieceInstance` class encapsulates a generator for a public and private key of the McEliece protocol and is constructed from the random Goppa code.
This class scrambles the generator into $SGP$ where $P$ is a random permutation matrix and $S$ is a random invertible matrix over $F_2$.
The method `public_private_pair()` generates a public key and private key tuple.
The public key contains an `encode` method which encodes any with the scrambled generator, while the private key contains a `decode` method using the known $S$ and $P$ matrices, which can correct up to weight $t$ errors.

In [24]:
import numpy as np
from mceliece.goppa import generate_random_goppa
from mceliece.generator import McElieceInstance
from mceliece.randutil import random_weight_t

rng = np.random.Generator(np.random.PCG64(1234))
# Generate a random goppa code
n=16
t=3
gc = generate_random_goppa(n, t, rng=rng)
# Sample a single random weight t error
err = np.reshape(random_weight_t(n, t, rng), (1, n))
# McEliece instance generator
instance = McElieceInstance(gc, rng)
# Generate the public/private
pub, priv = instance.public_private_pair()
# Generate a random plain-text message
mesg = rng.integers(0, 2, (1, gc.k), dtype=np.int8)
# Encode into ciphertext
y0 = pub.encode(mesg)
y = (y0 + err) % 2

In [27]:
print(mesg)
print(y0)

[[0 0 1 1]]
[[1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0]]


Since the RNG was used for random operations in the same order as pysa-genmci and used the same seed as above,
the plain-text and ciphertext are the same as the ones generated above and printed to `_plain.txt`. Similarly, we can check that the non-zero (one-based) indices of the public generator matrix transpose are written out to `_mld.txt`. Each line is signed when the corresponding `y[i]` is 1.

In [35]:
np.hstack([y.transpose(),pub.Gp.transpose()])

array([[1, 1, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0],
       [1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1],
       [0, 1, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1]], dtype=int8)

## Reductions to 3-SAT Instances

An additional product of `pysa-genmci` are reductions of the McEliece instances to CNF formatted SAT. Two types of reductions are provided: the `_xor_3sat.cnf` contains a reduction to mixed XORSAT/3-SAT clauses, while `_3sat.cnf` contains a reduction to 3-SAT clauses only.

The mixed clauses can be constructed by using the PCD formulation (i.e. the Niederreiter dual) of a McEliece instance, which reduces to finding a vector $e$ over $\mathbb{F}_2$ to the linear equation $z=He$ such that $e$ has a Hamming weight $\mathrm{wt}(e) = t$, where $H$ is the parity check matrix and $z$ is the syndrome of the ciphertext. The rows of the linear equation are simply XORSAT clauses, while the Hamming weight constraint can be enforced by constructing a Boolean circuit $\mathrm{ACC}(e)$ that computes $\mathrm{wt}(e)$.
By enforcing the requirement that the output bits of $\mathrm{ACC}(e)$ are the binary representation of the integer $t$, we have a CIRCUIT-SAT problem that is easily reduced to 3-SAT clauses via the Tseytin reduction. This circuit is constructed from $O(n\log n)$ half-adders. The self-contained Tseytin reduction for this circuit is implemented in `mceliece/tseytin/tseytin.h` and `tseytin.cpp`.

The function mceliece.tseytin.reduce.reduce_mld_to_sat performs the reduction from numpy arrays of $H$ and $z$, and returns the reduced problem as a string in XOR-extended DIMACS format (compatible with cryptominisat). Additional functions and data types in `tseytin.cpp` also have Python bindings available, but these are not yet stabilized.
The reduction can be performed as follows:


In [44]:
from mceliece.lincodes import dual_code
from mceliece.f2mat import f2_matmul
from mceliece.tseytin.reduce import reduce_mld_to_sat

# Evaluate the parity check matrix (i.e. the dual code)
Hdual = dual_code(pub.Gp)
# Evaluate the syndrome bits z
z = f2_matmul(y, Hdual.transpose())
# Get the DIMACS string of the reduced problem
xs_dimacs = reduce_mld_to_sat(Hdual, z[0], t, as_3sat_only=False)


The first few lines of the DIMACS string are:

In [45]:
print('\n'.join(xs_dimacs.splitlines()[:20]))

p cnf 114 371
x-1 4 0
x-2 6 0
x2 3 5 7 0
x-1 3 5 8 0
x-1 2 5 9 0
x-1 3 5 10 0
x-3 11 0
x-1 2 5 12 0
x-1 3 13 0
x2 5 14 0
x2 3 15 0
x-5 16 0
-1 -2 -17 0
1 2 -17 0
1 -2 17 0
-1 2 17 0
-1 -2 18 0
1 -18 0
2 -18 0


Which we can confirm is the reduced problem output by `pysa-genmci`.

In [47]:
! head -n20 ./mci_example_16-3_pub/mci_n16_t3_m4_k4_0_xor_3sat.cnf

p cnf 114 371
x-1 4 0
x-2 6 0
x2 3 5 7 0
x-1 3 5 8 0
x-1 2 5 9 0
x-1 3 5 10 0
x-3 11 0
x-1 2 5 12 0
x-1 3 13 0
x2 5 14 0
x2 3 15 0
x-5 16 0
-1 -2 -17 0
1 2 -17 0
1 -2 17 0
-1 2 17 0
-1 -2 18 0
1 -18 0
2 -18 0


Finally, the `_3sat.cnf` file replaces the XOR clauses with corresponding 3-SAT clauses, fully describing the instance in genuine 3-SAT format. Its DIMACS string is obtained by passing `as_3sat_only=True` to `reduce_mld_to_sat`. Note that this reduction has a more expensive proliferation of $O(n(n-k))$ auxiliary variables.